<img align="right" src="../../../additional_data/banner_siegel.png" style="width:1000px;">

# Exercise II for using xarray library 

* [**Sign up to the JupyterHub**](https://www.phenocube.org/) to run this notebook interactively from your browser
* **Compatibility:** Notebook currently compatible with the Open Data Cube environments of the University of Wuerzburg


## Load packages and connect to eo2cube datacube

Import following libraries; datacube and with_ui_cbk from odc.ui. And connect to eo2cube dataset. 

In [59]:
import datacube
from odc.ui import with_ui_cbk

dc = datacube.Datacube(app = "Exercise_for_xarrays")

## Load data with the dc.load() function 

Load Sentinel-2 data from the eo2cube datacube with dc.load() function and following options.<br> 

* Longitude range: (24.80, 24.95), <br> 
* Latitude range: (-28.88, -28.79) <br> 
* Time range: 2020-08-01 to 2020-10-31 <br> 
* Bands: blue, green, red, nir <br> 
* Output coordinate system: EPSG:32734 <br> 
* Resolution: 10m <br> 
* Select only solar days 
* Use loading bar


In [3]:
ds = dc.load(product= "s2_l2a",
               x = (24.80, 24.95),
               y = (-28.88, -28.79),
               output_crs = "EPSG:32734",
               time = ("2020-08-01", "2020-10-31"),
               measurements = ["blue", "green", "red","nir"],
               resolution = (-10,10),
               group_by = "solar_day",
               progress_cbk=with_ui_cbk())

ds

/opt/conda/lib/python3.9/site-packages/datacube/drivers/postgres/_api.py:564: SAWarning: Class CommonTimestamp will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  return self._connection.execute(select_query)
/opt/conda/lib/python3.9/site-packages/datacube/drivers/postgres/_api.py:564: SAWarning: Class Float8Range will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  

<xarray.Dataset>
Dimensions:      (time: 18, y: 1047, x: 1498)
Coordinates:
  * time         (time) datetime64[ns] 2020-08-02T08:28:17 ... 2020-10-31T08:...
  * y            (y) float64 6.809e+06 6.809e+06 ... 6.799e+06 6.799e+06
  * x            (x) float64 8.707e+05 8.707e+05 ... 8.857e+05 8.857e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 936 907 898 899 930 ... 7752 7696 7752 7804
    green        (time, y, x) uint16 1280 1246 1262 1278 ... 7080 7104 7172 7172
    red          (time, y, x) uint16 1808 1804 1804 1804 ... 6800 6728 6696 6704
    nir          (time, y, x) uint16 2522 2414 2448 2524 ... 6908 6940 6976 7044
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

## Index the dataset 

1. Select the dataset of first 4 time stamps and show the exact time stamps

In [4]:
ds.isel(time=slice(0,3)).time 
# or ds.isel(time=[0,1,2,3]).time

<xarray.DataArray 'time' (time: 3)>
array(['2020-08-02T08:28:17.000000000', '2020-08-07T08:28:20.000000000',
       '2020-08-12T08:28:17.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-08-02T08:28:17 ... 2020-08-12T08:...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00

2. Slice the dataset for September 2020

In [5]:
ds.sel(time=slice("2020-09-01","2020-09-30"))

<xarray.Dataset>
Dimensions:      (time: 6, y: 1047, x: 1498)
Coordinates:
  * time         (time) datetime64[ns] 2020-09-01T08:28:17 ... 2020-09-26T08:...
  * y            (y) float64 6.809e+06 6.809e+06 ... 6.799e+06 6.799e+06
  * x            (x) float64 8.707e+05 8.707e+05 ... 8.857e+05 8.857e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 1306 1262 1282 1260 ... 1228 1270 1224 1268
    green        (time, y, x) uint16 1752 1726 1742 1758 ... 1954 2002 1972 1992
    red          (time, y, x) uint16 2306 2290 2324 2370 ... 2776 2834 2824 2834
    nir          (time, y, x) uint16 3108 3078 3066 3162 ... 3544 3588 3564 3574
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

## Data Manipulation & Statistics

1. print the maximum value of near infrared band.

In [45]:
print(ds.nir.max())

<xarray.DataArray 'nir' ()>
array(16160, dtype=uint16)
Coordinates:
    spatial_ref  int32 32734


2. print the mean value of near infrared band.

In [46]:
print(ds.nir.mean())

<xarray.DataArray 'nir' ()>
array(3163.83899967)
Coordinates:
    spatial_ref  int32 32734


3. Find the reflectance values that are greater than 1000 in the Near Infrared band using `where` function.

In [56]:
print(ds.where(ds.nir > 1000))

<xarray.Dataset>
Dimensions:      (time: 18, y: 1047, x: 1498)
Coordinates:
  * time         (time) datetime64[ns] 2020-08-02T08:28:17 ... 2020-10-31T08:...
  * y            (y) float64 6.809e+06 6.809e+06 ... 6.799e+06 6.799e+06
  * x            (x) float64 8.707e+05 8.707e+05 ... 8.857e+05 8.857e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) float64 936.0 907.0 898.0 ... 7.752e+03 7.804e+03
    green        (time, y, x) float64 1.28e+03 1.246e+03 ... 7.172e+03 7.172e+03
    red          (time, y, x) float64 1.808e+03 1.804e+03 ... 6.704e+03
    nir          (time, y, x) float64 2.522e+03 2.414e+03 ... 7.044e+03
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref


4. Calculate the mean of the monthly grouped dataset. 

In [58]:
ds.groupby("time.month").mean(dim="time")

<xarray.Dataset>
Dimensions:      (y: 1047, x: 1498, month: 3)
Coordinates:
  * y            (y) float64 6.809e+06 6.809e+06 ... 6.799e+06 6.799e+06
  * x            (x) float64 8.707e+05 8.707e+05 ... 8.857e+05 8.857e+05
    spatial_ref  int32 32734
  * month        (month) int64 8 9 10
Data variables:
    blue         (month, y, x) float64 921.8 905.2 920.8 ... 2.469e+03 2.47e+03
    green        (month, y, x) float64 1.342e+03 1.321e+03 ... 2.93e+03
    red          (month, y, x) float64 1.889e+03 1.864e+03 ... 3.566e+03
    nir          (month, y, x) float64 2.587e+03 2.536e+03 ... 4.214e+03

***

## Additional information

<font size="2">This notebook for the usage in the Open Data Cube entities of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/), is adapted from [Geoscience Australia](https://github.com/GeoscienceAustralia/dea-notebooks), published using the Apache License, Version 2.0. Thanks! </font>

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** June 2022